## AI Finance Project - Loda Enrico

In [3]:
# import
import pandas as pd
import yfinance as yf

We want to calculate the daily simple rate of return of our assets:




**Daily simple rate of return :**
$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_t}{P_{t-1}} - 1
$

In [4]:
df = yf.download(["SPY","NVDA"], start="2010-01-01", end="2025-01-01")
spy = df.xs('SPY', axis=1, level=1)
x = df.xs('NVDA', axis=1, level=1)

prices = df["Close"]
returns = (prices / prices.shift(1)) - 1
returns = returns.fillna(0)
SPY_r = returns['SPY']
NVDA_r = returns['NVDA']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


In [22]:
df

Price            Close                    High                     Low  \
Ticker            NVDA         SPY        NVDA         SPY        NVDA   
Date                                                                     
2010-01-04    0.423884   85.768448    0.426864   85.813854    0.415172   
2010-01-05    0.430073   85.995506    0.434658   86.033349    0.422279   
2010-01-06    0.432824   86.056007    0.433741   86.267911    0.425718   
2010-01-07    0.424342   86.419289    0.432366   86.525241    0.421133   
2010-01-08    0.425259   86.706886    0.428239   86.744729    0.418382   
...                ...         ...         ...         ...         ...   
2024-12-24  140.207108  599.496582  141.886946  599.536501  138.637245   
2024-12-26  139.917130  599.536499  140.837058  600.673034  137.717335   
2024-12-27  136.997391  593.225464  139.007216  595.987176  134.697615   
2024-12-30  137.477356  586.455811  140.257099  589.965273  134.007674   
2024-12-31  134.277649  584.322266  138.057315  588.868587  133.817700   

Price                         Open                 Volume             
Ticker             SPY        NVDA         SPY       NVDA        SPY  
Date                                                                  
2010-01-04   84.391067    0.424342   85.041918  800204000  118944600  
2010-01-05   85.405201    0.422279   85.715493  728648000  111579900  
2010-01-06   85.844104    0.429844   85.912213  649168000  116074400  
2010-01-07   85.654916    0.430532   85.897093  547792000  131091100  
2010-01-08   86.018199    0.420903   86.192260  478168000  126402800  
...                ...         ...         ...        ...        ...  
2024-12-24  593.684050  139.987127  594.272307  105157000   33160100  
2024-12-26  596.286267  139.687155  597.701991  116205600   41219100  
2024-12-27  588.988210  138.537258  595.747844  170582600   64969300  
2024-12-30  582.657240  134.817597  586.126844  167734700   56578800  
2024-12-31  582.667210  138.017311  588.140735  155659200   57052700  

[3774 rows x 10 columns]

In [11]:
returns[:]

Ticker,NVDA,SPY
Date,,
2010-01-04,0.000000,0.000000
2010-01-05,0.014603,0.002647
2010-01-06,0.006396,0.000704
2010-01-07,-0.019598,0.004221
2010-01-08,0.002161,0.003328
...,...,...
2024-12-24,0.003938,0.011115
2024-12-26,-0.002068,0.000067
2024-12-27,-0.020868,-0.010527


#dataset for training
The features, for a total of 24, are the following:

*   $\mu_1$, $\mu_2$ of $r_1$, $r_2$ calculated on the last 10 lags
*   $[r_1^t,\dots ,r_1^{t-10}]$, $[r_2^t,\dots ,r_2^{t-10}]$

The target is the values of my covariance matrix:
* $\Sigma_{t+1}$